In [4]:
import nltk, os, glob
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import pandas as pd
import re
from num2words import num2words
import plotly.graph_objects as go
import import_ipynb

nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\c1635922\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [6]:
# Function to take file path and return path object which will get all files of that type in the directory.
# Takes two param: filename and filetype.
def file_to_path(file_dir, filetype):
    # Finds all files of the specified type in the directory
    txt_path = os.path.join(file_dir, '*.' + filetype)

    # Uses Glob with the txt_path variable to place all the file directories in a list
    file_list = glob.glob(txt_path)

    return file_list

In [10]:
# Function to create corpus
def create_corpus(path):
    token_corpus = {}
    for f in path:
        with open(f, 'r', encoding="utf-8") as fs:
                text = fs.read()
                text = text.replace('\n', ' ')
                #text = prepro.preprocess(text)
                token_corpus[f] = text.replace('\n', ' ')
    return token_corpus

In [11]:
# Function to create comparison file
def create_comparison_str(path):
    token_file = ""

    for f in path:
        with open(f, 'r', encoding="utf-8") as fs:
            text = fs.read()
            text = text.replace('\n', ' ')
            #text = prepro.preprocess(text)
            token_file += text
    return token_file

In [12]:
corpus_files = file_to_path('../txts/individual-days/', 'txt')
token_corpus = create_corpus(corpus_files)

In [13]:
comparison_files = file_to_path('../txts/Pronoun/', 'txt')
token_file = create_comparison_str(comparison_files)

In [14]:
# tranform pronoun file
def transform_each_pronoun_doc():
    all_pronoun_files_and_text = {}
    queries = ["я", "мы", "ты", "нас", "вы", "тебя", "меня", "тебе", "мне"]

    for query in queries:
        all_pronoun_files_and_text[query] = save_file_string(query)

    return all_pronoun_files_and_text

def save_file_string(query):
    base_path = '../txts/Pronoun/' + query
    path_to_files = file_to_path(base_path, 'txt')
    return create_comparison_str(path_to_files)

pronoun_dictionary = transform_each_pronoun_doc()

In [15]:
token_я = pronoun_dictionary.get("я")
token_мы = pronoun_dictionary.get("мы")
token_ты = pronoun_dictionary.get("ты")
token_нас = pronoun_dictionary.get("нас")
token_вы = pronoun_dictionary.get("вы")
token_тебя = pronoun_dictionary.get("тебя")
token_меня = pronoun_dictionary.get("меня")
token_тебе = pronoun_dictionary.get("тебе")
token_мне = pronoun_dictionary.get("мне")

In [16]:
# https://towardsdatascience.com/higher-accuracy-and-less-process-time-in-text-classification-with-lda-and-tf-idf-d2d949e344c3
# https://www.bogotobogo.com/python/NLTK/tf_idf_with_scikit-learn_NLTK.php
# https://www.datacamp.com/community/tutorials/stemming-lemmatization-python

# The transform() function computes the tfidf frequency of each word in the bag of word.
# Now our aim is to compare the document D2 with D1. It means we want to see how many words
# of D1 match up with D2. Thats why we perform fit_transform() on D1 and then only the transform()
# function on D2 would apply the bag of words of D1 and count the inverse frequency of tokens in D2.
# This would give the relative comparison of D1 against D2.

tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=8)

tfidf_matrix = tfidf.fit_transform(token_corpus.values())
feature_names = tfidf.get_feature_names()

response_я = tfidf.transform([token_я])
response_мы= tfidf.transform([token_мы])
response_ты = tfidf.transform([token_ты])
response_нас = tfidf.transform([token_нас])
response_вы = tfidf.transform([token_вы])
response_тебя = tfidf.transform([token_тебя])
response_меня = tfidf.transform([token_меня])
response_тебе = tfidf.transform([token_тебе])
response_мне = tfidf.transform([token_мне])

# TFIDF

In [17]:
# lst = token_мы.split()
# for i in lst:
#     if len(i) == 2:# == "ве":
#         print(i)

print(response_я)

  (0, 13191)	0.000881236878422398
  (0, 13190)	0.0002885325190712215
  (0, 13189)	0.0005401655165393814
  (0, 13188)	0.0044265624076819665
  (0, 13185)	0.004593220505972547
  (0, 13184)	0.0013234798385213715
  (0, 13183)	0.0002956447003864589
  (0, 13181)	0.0002885325190712215
  (0, 13180)	0.0012740651631062937
  (0, 13179)	0.0021830114302204117
  (0, 13178)	0.002084932822315402
  (0, 13177)	0.0022451764562953186
  (0, 13175)	0.001556687171907707
  (0, 13174)	0.0007021656730449253
  (0, 13171)	0.0014881821029542846
  (0, 13170)	0.0017291562492789858
  (0, 13169)	0.0023934798290990733
  (0, 13168)	0.0023244573822296926
  (0, 13167)	0.0015037106197026745
  (0, 13166)	0.0005401655165393814
  (0, 13165)	0.001154130076284886
  (0, 13164)	0.0003118448516217685
  (0, 13163)	0.006584284437695411
  (0, 13162)	0.0009355345548653055
  (0, 13161)	0.00913717848686233
  :	:
  (0, 55)	0.009226758668106077
  (0, 53)	0.0006424132850064543
  (0, 51)	0.0006937740022114252
  (0, 46)	0.0005830541585135155


In [23]:
import numpy as np

def tfidf_results(item, pronoun):
    data = {}
    words = []
    count = []

    for col in item.nonzero()[1]:
        if feature_names[col] != "ве":
            words.append(feature_names[col])
            count.append(item[0, col])

        #word_column = "Word" + " " + pronouns[i]
        #tfidf_column = "TF-IDF" + " " + pronouns[i]

    for x in range(len(words)):
            data["Words-"+pronoun] = words
            data["TF-IDF-"+pronoun] = count

    df = pd.DataFrame(data=data)
    queries = ["я", "мы", "ты", "нас", "вы", "тебя", "меня", "тебе", "мне"]

    df_sorted = df.sort_values(by=["TF-IDF-"+pronoun], ascending=False).reset_index(drop=True)
    top = df_sorted.head(6)
    return top.drop([0])

я_pro = tfidf_results(response_я, "я")
мы_pro = tfidf_results(response_мы, "мы")
ты_pro = tfidf_results(response_ты, "ты")
нас_pro = tfidf_results(response_нас, "нас")
вы_pro = tfidf_results(response_вы, "вы")
тебя_pro = tfidf_results(response_тебя, "тебя")
меня_pro = tfidf_results(response_меня, "меня")
тебе_pro = tfidf_results(response_тебе, "тебе")
мне_pro = tfidf_results(response_мне, "мне")

result = pd.concat([я_pro, мы_pro, ты_pro, нас_pro, вы_pro, тебя_pro, меня_pro, тебе_pro, мне_pro], axis=1)

column_values = result[["Words-я", "Words-мы", "Words-ты", "Words-нас", "Words-вы", "Words-тебя","Words-меня", "Words-тебе", "Words-мне"]].values
unique_values =  np.unique(column_values)

column_value_count = result[["Words-я", "Words-мы", "Words-ты", "Words-нас", "Words-вы", "Words-тебя", "Words-меня","Words-тебе", "Words-мне"]].value_counts()
unique_values_count =  np.unique(column_value_count)

display(result)

,Words-я,TF-IDF-я,Words-мы,TF-IDF-мы,Words-ты,TF-IDF-ты,Words-нас,TF-IDF-нас,Words-вы,TF-IDF-вы,Words-тебя,TF-IDF-тебя,Words-меня,TF-IDF-меня,Words-тебе,TF-IDF-тебе,Words-мне,TF-IDF-мне
1,человек,0.232178,человек,0.215112,человек,0.236633,человек,0.201346,человек,0.269101,тво,0.258267,человек,0.236742,человек,0.231846,человек,0.247365
2,моч,0.179664,наш,0.203139,сво,0.197972,наш,0.157783,ваш,0.200418,человек,0.234945,котор,0.179488,тво,0.189500,котор,0.177318
3,прост,0.164381,котор,0.189513,моч,0.190973,моч,0.146096,моч,0.175929,моч,0.225443,моч,0.167179,сво,0.182089,прост,0.161198
4,котор,0.162160,моч,0.172998,тво,0.180974,котор,0.136002,сво,0.174522,котор,0.169298,прост,0.164889,моч,0.155623,моч,0.153284
5,знат,0.151142,знат,0.158134,знат,0.147979,ход,0.129095,котор,0.168892,сво,0.164980,оказыва,0.153152,говор,0.142919,каза,0.143613


In [19]:
# https://towardsdatascience.com/higher-accuracy-and-less-process-time-in-text-classification-with-lda-and-tf-idf-d2d949e344c3
# https://www.bogotobogo.com/python/NLTK/tf_idf_with_scikit-learn_NLTK.php
# https://www.datacamp.com/community/tutorials/stemming-lemmatization-python

# The transform() function computes the tfidf frequency of each word in the bag of word.
# Now our aim is to compare the document D2 with D1. It means we want to see how many words
# of D1 match up with D2. Thats why we perform fit_transform() on D1 and then only the transform()
# function on D2 would apply the bag of words of D1 and count the inverse frequency of tokens in D2.
# This would give the relative comparison of D1 against D2.

tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=8, stop_words = 'english' )

tfidf_matrix = tfidf.fit_transform(token_corpus.values())
feature_names = tfidf.get_feature_names()

response_я = tfidf.transform([token_я])
response_мы = tfidf.transform([token_мы])
response_ты  = tfidf.transform([token_ты])
response_нас = tfidf.transform([token_нас])
response_вы = tfidf.transform([token_вы])
response_тебя = tfidf.transform([token_тебя])
response_меня = tfidf.transform([token_меня])
response_тебе = tfidf.transform([token_тебе])
response_мне = tfidf.transform([token_мне])

In [20]:
# Size of corpus
print(tfidf_matrix.shape)
df = pd.DataFrame(tfidf_matrix.toarray(), columns = tfidf.get_feature_names())

(122, 13192)


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer

import pandas as pd
import plotly.graph_objects as go

# Links used for understanding tfidf
# https://towardsdatascience.com/higher-accuracy-and-less-process-time-in-text-classification-with-lda-and-tf-idf-d2d949e344c3
# https://www.bogotobogo.com/python/NLTK/tf_idf_with_scikit-learn_NLTK.php
# https://www.datacamp.com/community/tutorials/stemming-lemmatization-python

corpus = files_to_corpus()

# TfidfVect object with variables specified.
tfidf = TfidfVectorizer(tokenizer=tokenize, analyzer='word',
                        sublinear_tf=True, smooth_idf=True, use_idf=True)

# Forms a tfidf Matrix using the text from the files.
tfidf_matrix = tfidf.fit_transform(corpus.values())
feature_names = tfidf.get_feature_names()

#  
for i in range(len(corpus)):
    feature_index = tfidf_matrix[i,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])

# sum tfidf frequency of each term through documents
sums = tfidf_matrix.sum(axis=0)

# connecting term to its sum frequency
data = []
for col, term in enumerate(feature_names):
    if col > 1000 and len(term) > 3:
        data.append( (str(posTag(term)), sums[0,col] ))

NameError: name 'files_to_corpus' is not defined

In [ ]:
# Saves to CSV
compression_opts = dict(method='zip',
                        archive_name='out.csv')  

ordering.to_csv('out.zip', index=False,
          compression=compression_opts)

Read SCHEMA of json file

In [ ]:
import pandas as pd

# Open json, print json schema
df_idf = pd.read_json("../russianDataSets/ru/2020-03-01.json",lines=True)

# print schema
print("Schema:\n\n",df_idf.dtypes)
print("Number of questions,columns=",df_idf.shape)

Schema:

 quote_count                                int64
contributors                             float64
truncated                                   bool
text                                      object
is_quote_status                             bool
in_reply_to_status_id                    float64
reply_count                                int64
id                                         int64
favorite_count                             int64
created_at_src                            object
collected_by                              object
retweeted                                   bool
investigationId                            int64
coordinates                               object
timestamp_ms                      datetime64[ns]
entities                                  object
in_reply_to_screen_name                   object
id_str                                     int64
text_translation                          object
retweet_count                              int64
stored_at 

In [ ]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    if text[2] == "rt":
        text=""
    #remove tags
    text=re.sub("","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

#load a set of stop words
stopwords=get_stop_words("../txts/2020-03-01.txt")

#get the text column 
docs=df_idf['text'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000)
word_count_vector=cv.fit_transform(docs)
list(cv.vocabulary_.keys())[:10]

C:\Users\c1635922\AppData\Roaming\Python\Python38\site-packages\sklearn\feature_extraction\text.py:388: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['12', '13', '1400', '1595', '1600', '2008', '22', '2выздоровевшие', 'авто', 'астроном', 'блокчейн', 'вице', 'военно', 'военной', 'вторых', 'гей', 'демократа', 'док', 'доказательство', 'другому', 'душит', 'дь', 'европейска', 'женщину', 'журналистике', 'заболевшие', 'заек', 'захватчк', 'избавление', 'израиль', 'квартиры', 'коллега', 'коммунизм', 'крови', 'летняя', 'лужаек', 'марино', 'морских', 'морской', 'наркомана', 'недоразумение', 'нибудь', 'нон', 'ов', 'онлайн', 'оружии', 'отдохни', 'пекински', 'первых', 'пид', 'погибшие', 'подруга', 'порно', 'прежнему', 'премьер', 'прерогатива', 'пресс', 'причинно', 'пропаганда', 'рмот', 'салоном', 'сан', 'сканворды', 'следственных', 'службы', 'спаса', 'стопом', 'су', 'тату', 'тихо', 'толку', 'укро', 'чудо', 'чью', 'штаб', 'штирл

['rt',
 'shining_stri',
 'coldabbess',
 'тёмная',
 'даёт',
 'плю',
 'https',
 'co',
 'oubahqve',
 'vovablya']

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [ ]:
# read test docs into a dataframe and concatenate title and body
df_test=pd.read_json("../russianDataSets/ru/2020-03-02.json",lines=True)
df_test['text'] =df_test['text'].apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=df_test['text'].tolist()